In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy
from multiprocessing import Pool, shared_memory

In [2]:
mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
database = f"/home/jeon/MissingSat/database"

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [3]:
LG = pklload(f"{database}/00_LocalGroup_final.pickle")
filedone = []
if(os.path.exists(f"{database}/00_LocalGroup_final_a_realsubs.pickle")):
    LG = pklload(f"{database}/00_LocalGroup_final_a_realsubs.pickle")
    filedone.append('a')
if(os.path.exists(f"{database}/00_LocalGroup_final_b_realsats.pickle")):
    LG = pklload(f"{database}/00_LocalGroup_final_b_realsats.pickle")
    filedone.append('b')
if(os.path.exists(f"{database}/00_LocalGroup_final_c_realreal.pickle")):
    LG = pklload(f"{database}/00_LocalGroup_final_c_realreal.pickle")
    filedone.append('c')
if(os.path.exists(f"{database}/00_LocalGroup_final_d_realparking.pickle")):
    LG = pklload(f"{database}/00_LocalGroup_final_d_realparking.pickle")
    filedone.append('d')
if(os.path.exists(f"{database}/00_LocalGroup_final_e_removeunreal.pickle")):
    LG = pklload(f"{database}/00_LocalGroup_final_e_removeunreal.pickle")
    filedone.append('e')
if(os.path.exists(f"{database}/00_LocalGroup_final_f_addcat.pickle")):
    LG = pklload(f"{database}/00_LocalGroup_final_f_addcat.pickle")
    filedone.append('f')


virials = pklload(f"{database}/virial_radius_nh_1026.pickle")

# Subs

In [4]:
dtype1 = hals.dtype
dtype2 = virials.dtype
dtype = np.dtype(dtype1.descr + dtype2.descr)
nhals = np.zeros(hals.shape, dtype=dtype)
for iname in tqdm(hals.dtype.names):
    nhals[iname] = hals[iname]
for iname in tqdm(virials.dtype.names):
    nhals[iname] = virials[iname]
print(dtype)

100%|██████████| 3/3 [00:00<00:00, 6612.15it/s]

[('nparts', '<i4'), ('id', '<i4'), ('timestep', '<i4'), ('level', '<i4'), ('host', '<i4'), ('hostsub', '<i4'), ('nbsub', '<i4'), ('nextsub', '<i4'), ('aexp', '<f8'), ('m', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Lx', '<f8'), ('Ly', '<f8'), ('Lz', '<f8'), ('r', '<f8'), ('a', '<f8'), ('b', '<f8'), ('c', '<f8'), ('ek', '<f8'), ('ep', '<f8'), ('et', '<f8'), ('spin', '<f8'), ('sigma', '<f8'), ('rvir', '<f8'), ('mvir', '<f8'), ('tvir', '<f8'), ('cvel', '<f8'), ('rho0', '<f8'), ('rc', '<f8'), ('mcontam', '<f8'), ('r200kpc', '<f8'), ('m200', '<f8'), ('r200', '<f8')]


In [5]:
if(not 'a' in filedone):
    hor='x'; ver='y'
    show = False
    for key in LG.keys():
        print(f"[{key:04d}]")
        BGG = LG[key]['BGG']
        sats = LG[key]['sats']
        ihals = cut_sphere(nhals, BGG['x'], BGG['y'], BGG['z'], 1.5*BGG['r200_code'], both_sphere=True, rname='r200')
        ihals = ihals[ihals['id'] != BGG['halo_id']]

        if(show):
            fig, ax = fancy_axis(figsize=(5, 5), dpi=50)
            ax.scatter(ihals[hor], ihals[ver], s=0.1)
            for ihal in ihals:
                cir = circle(ihal, rname='r200', color='w', hor=hor, ver=ver)
                ax.add_patch(cir)
            cir = circle(BGG, rname='r200_code', color='yellow', hor=hor, ver=ver)
            ax.add_patch(cir)
        done = np.array([BGG['halo_id']])
        queue = ihals['id']
        lenq = len(queue)
        i = 0
        print(f"{i:04d}th Nremain={len(queue):04d}")
        while(len(queue)>0):
            ihal = nhals[queue[0]-1]
            queue = queue[1:]
            done = np.append(done, ihal['id'])

            tmp = cut_sphere(nhals, ihal['x'], ihal['y'], ihal['z'], 1.5*ihal['r200'], both_sphere=True, rname='r200', factor=1.5)
            tmp = tmp[~isin(tmp['id'], done)]
            queue = np.append(queue, tmp['id'])
            queue = np.unique(queue)
            i += 1
            if(len(queue) < lenq/2):
                # print(f"{i:04d}th Nremain={len(queue):04d}")
                lenq = len(queue)
        news = done[~isin(done, ihals['id'])]
        news = news[news != BGG['halo_id']]
        print(f"Total {len(news)} new halos found")
        LG[key]['Final_subs'] = done[done != BGG['halo_id']]
        if(show):
            for new in news:
                ihal = nhals[new-1]
                cir = circle(ihal, rname='r200', color='cyan', hor=hor, ver=ver)
                ax.add_patch(cir)
            
            for sat in sats:
                if(sat['id']<0):
                    cir = circle(sat, rname='r', color='magenta', ls=':', hor=hor, ver=ver)
                else:
                    cir = circle(sat, rname='r', color='magenta', hor=hor, ver=ver)
                ax.add_patch(cir)
            ax.set_aspect(1)
            plt.show()
            plt.close()
        # break

    del LG[92]
    pklsave(LG, f"{database}/00_LocalGroup_final_a_realsubs.pickle", overwrite=True)

I found all subs in `92` are included in `4`

1 3 7 8 10 23 25 26 36 64 121 129 11
1 3 7 8 10 23 25 26 36 64 121 129 11

# Sats

In [6]:
if(not 'b' in filedone):
    hor='x'; ver='y'; show=False
    for key in LG.keys():
        print(f"[{key:04d}]")
        BGG = LG[key]['BGG']
        if(show):
            fig, ax = fancy_axis(figsize=(5, 5), dpi=50)
            cir = circle(BGG, rname='r200_code', color='yellow', hor=hor, ver=ver, zorder=10)
            ax.add_patch(cir)

        x1=BGG['x'] - 1.5*BGG['r200_code']; x2=BGG['x'] + 1.5*BGG['r200_code']
        y1=BGG['y'] - 1.5*BGG['r200_code']; y2=BGG['y'] + 1.5*BGG['r200_code']
        z1=BGG['z'] - 1.5*BGG['r200_code']; z2=BGG['z'] + 1.5*BGG['r200_code']
        subids = LG[key]['Final_subs']
        subs = nhals[subids-1]
        if(show): ax.scatter(subs['x'], subs['y'], s=0.1, c='w')
        x1 = min(x1, np.min(subs['x']-1.5*subs['r200']))
        x2 = max(x2, np.max(subs['x']+1.5*subs['r200']))
        y1 = min(y1, np.min(subs['y']-1.5*subs['r200']))
        y2 = max(y2, np.max(subs['y']+1.5*subs['r200']))
        z1 = min(z1, np.min(subs['z']-1.5*subs['r200']))
        z2 = max(z2, np.max(subs['z']+1.5*subs['r200']))
        
        indx1 = (gals['x']+gals['r']) >= x1; indx2 = (gals['x']-gals['r']) <= x2
        indy1 = (gals['y']+gals['r']) >= y1; indy2 = (gals['y']-gals['r']) <= y2
        indz1 = (gals['z']+gals['r']) >= z1; indz2 = (gals['z']-gals['r']) <= z2
        igals = gals[indx1 & indx2 & indy1 & indy2 & indz1 & indz2]
        igals = igals[igals['id'] != BGG['id']]
        if(show):
            for sub in tqdm( subs ):
                cir = circle(sub, rname='r200', color='w', hor=hor, ver=ver)
                ax.add_patch(cir)
            ax.scatter(igals['x'], igals['y'], s=1, c='cyan')
            for igal in tqdm( igals ):
                cir = circle(igal, rname='r', color='cyan', hor=hor, ver=ver)
                ax.add_patch(cir)
            plt.show()
            plt.close()
        # break
        LG[key]['Final_sats'] = igals['id']
    pklsave(LG, f"{database}/00_LocalGroup_final_b_realsats.pickle", overwrite=True)

# Matching

In [7]:
def point_in_sphere(point, sphere, rname='r', factor=1):
    dist = np.sqrt( (point['x'] - sphere['x'])**2 + (point['y'] - sphere['y'])**2 + (point['z'] - sphere['z'])**2 )
    # print(dist, sphere[rname]*factor)
    return dist < sphere[rname]*factor

def sphere_in_sphere(inner, outer, r1='r',r2='r', factor=1):
    dist = np.sqrt( (inner['x'] - outer['x'])**2 + (inner['y'] - outer['y'])**2 + (inner['z'] - outer['z'])**2 )
    # print(dist+inner[r1], outer[r2])
    return (dist+inner[r1]) < outer[r2]

def sphere_touch_sphere(sph1, sph2, r1='r',r2='r', factor=1):
    dist = np.sqrt( (sph1['x'] - sph2['x'])**2 + (sph1['y'] - sph2['y'])**2 + (sph1['z'] - sph2['z'])**2 )
    # print(dist, sph1[r1]+sph2[r2])
    return dist < (sph1[r1]+sph2[r2])*factor


In [8]:
if(not 'c' in filedone):
    for key in LG.keys():
        BGG = LG[key]['BGG']
        sats = LG[key]['Final_sats']
        sats = gals[sats-1]; nsat = len(sats)
        subs = LG[key]['Final_subs']
        subs = nhals[subs-1]; nsub = len(subs)
        
        real = LG[key]['real']
        nreal = np.zeros(len(subs)+len(sats), dtype=real.dtype)
        nreal['gid'] = -1; nreal['hid'] = -1; nreal['state'] = 'dink'
        nreal[:nsub]['hid'] = subs['id']

        isats = sats
        paired_sats = []
        count1 = 0; count2 = 0
        for i in tqdm( range(nsub) ):
            isats = sats[~isin(sats['id'], paired_sats)]
            if(subs[i]['id'] in real['hid']):
                where = np.where(real['hid'] == nreal['hid'][i])[0]
                if(real[where]['gid']>0):
                    gid = real[where]['gid']
                    if(gid in isats['id']):
                        nreal[i]['gid'] = gid
                        nreal[i]['state'] = 'pair'
                        count1 +=1; paired_sats.append(gid)
                        continue
            insides2 = isats['r'] < subs[i]['r']
            insides = point_in_sphere(subs[i], isats, rname='r', factor=1) & insides2
            if(np.sum(insides)==0):
                insides = sphere_in_sphere(isats, subs[i], r2='rvir')
            if(np.sum(insides)==0):
                insides1 = sphere_touch_sphere(subs[i], isats, r1='rvir', factor=0.5)
                insides = insides1&insides2
            gcands = isats[insides]
            if(len(gcands)==0): continue
            elif(len(gcands)==1):
                gcand = gcands[0]
                nreal[i]['gid'] = gcand['id']; count2+=1
                nreal[i]['state'] = 'pair'
                paired_sats.append(gcand['id'])
            else:
                dists = distance(gcand, gcands)
                argmin = np.argmin(dists)
                gcand = gcands[argmin]
                nreal[i]['gid'] = gcand['id']; count2+=1
                nreal[i]['state'] = 'pair'
                paired_sats.append(gcand['id'])
            
        unique, counts = np.unique(paired_sats, return_counts=True)
        remained = sats[~isin(sats['id'], paired_sats)]
        nreal[nsub:nsub+len(remained)]['gid'] = remained['id']
        nreal[nsub:nsub+len(remained)]['state'] = 'orphan'
        assert np.all(counts==1)
        print(key, nsub, nsat, count1, count2, np.sum(real['state']=='pair'))
        mask = (nreal['gid']>0) | (nreal['hid']>0)
        nreal = nreal[mask]
        LG[key]['Final_real'] = nreal
    pklsave(LG, f"{database}/00_LocalGroup_final_c_realreal.pickle", overwrite=True)

# Remove Low Score

In [9]:
if(not 'd' in filedone):
    scores = pklload(f"{database}/befo231031/08_nh_scores.pickle")['take']
    tmp = pklload(f"./befo231109/recalc11/08_nh_take_scores_host0011.pickle")
    scores[11] = tmp

    dm_scores = pklload(f"{database}/befo231031/08_nh_dm_scores.pickle")['take']
    tmp = pklload(f"./befo231109/recalc11/08_nh_take_dm_scores_host0011.pickle")
    dm_scores[11] = tmp
    for key in scores.keys():
        if(os.path.exists(f"./befo231109/dontmiss/nh_take_dm_scores_host{key:04d}_new.pickle")):
            tmp = pklload(f"./befo231109/dontmiss/nh_take_dm_scores_host{key:04d}_new.pickle")
            for jkey in tmp.keys():
                if(jkey in dm_scores[key].keys()):
                    assert np.all(dm_scores[key][jkey] == tmp[jkey])
                    continue
                dm_scores[key][jkey] = tmp[jkey]

## Find omitted Sat & Sub

In [10]:
if(not 'd' in filedone):
    for key in LG.keys():
        BGG = LG[key]['BGG']
        print(f"[{key:04d}]")
        sats = LG[key]['Final_sats']
        sats = gals[sats-1]; nsat = len(sats)
        subs = LG[key]['Final_subs']
        subs = nhals[subs-1]; nsub = len(subs)
        real = LG[key]['Final_real']
        LG[key]['gparking'] = []
        LG[key]['hparking'] = []

        sat_scores = scores[key]
        gisin = isin(sats['id'], list(sat_scores.keys()))
        LG[key]['gparking'] = sats[~gisin]

        sub_scores = dm_scores[key]
        hisin = isin(subs['id'], list(sub_scores.keys()))
        LG[key]['hparking'] = subs[~hisin]
        print(f"Sat: {np.sum(gisin)}/{len(sats)} Sub: {np.sum(hisin)}/{len(subs)}")

    pklsave(LG, f"{database}/00_LocalGroup_final_d_realparking.pickle", overwrite=True)

In [11]:
pouts = snaps.iout_avail['iout'][snaps.iout_avail['age'] >= snap.age-1]
pouts = pouts[pouts < snap.iout][::-1]

def get_members(gal, galaxy=True):
    global members, snaps
    if(gal['timestep'] in members.keys()):
        if(gal['id'] in members[gal['timestep']].keys()):
            return members[gal['timestep']][gal['id']]
    else:
        members[gal['timestep']] = {}
    members[gal['timestep']][gal['id']] = uhmi.HaloMaker.read_member_part(snaps.get_snap(gal['timestep']), gal['id'], galaxy=galaxy, simple=True)
    return members[gal['timestep']][gal['id']]

In [12]:
if(not 'e' in filedone):
    for key in LG.keys():
        if(os.path.exists(f"./nh_take_dm_scores_host{key:04d}_new.pickle")): continue
        iLG = LG[key]
        BGG = iLG['BGG']
        rrange = BGG['r']#, BGG['r200_code'])

        gparking = iLG['gparking']
        if(len(gparking)>0):
            uri.timer.verbose=0
            centers = {}
            members = {}
            all_scores = {}
            give_scores = {}
            take_scores = {}
            for ip, pout in tqdm( enumerate(pouts), total=len(pouts), desc=f"G[{key:04}]" ):
                psnap = snaps.get_snap(pout)
                pgals = uhmi.HaloMaker.load(psnap, galaxy=True)

                for sat in gparking:
                    if(sat['id'] in centers.keys()):
                        center = centers[sat['id']]
                    else:
                        center = [sat['x'], sat['y'], sat['z']]
                        centers[sat['id']] = center
                    my_member = get_members(sat, galaxy=True)
                    pneighbors = cut_box(pgals, *center, rrange)


                    if(len(pneighbors)==0): continue

                    give_score = np.zeros(len(pneighbors))
                    take_score = np.zeros(len(pneighbors))
                    for i, pg in enumerate(pneighbors):
                        pmember = get_members(pg, galaxy=True)
                        intersect = np.sum( isin(pmember, my_member, assume_unique=True) )
                        give_score[i] = intersect / len(my_member) / 2
                        take_score[i] = intersect / len(pmember) / 2
                    all_score = give_score * take_score
                    
                    argmax_all = np.argmax(all_score)
                    argmax_give = np.argmax(give_score)
                    argmax_take = np.argmax(take_score)
                    if(not sat['id'] in all_scores.keys()):
                        all_scores[sat['id']] = np.zeros(len(pouts))
                        give_scores[sat['id']] = np.zeros(len(pouts))
                        take_scores[sat['id']] = np.zeros(len(pouts))
                    
                    all_scores[sat['id']][ip] = pneighbors['id'][argmax_all] + all_score[argmax_all]
                    give_scores[sat['id']][ip] = pneighbors['id'][argmax_give] + give_score[argmax_give]
                    take_scores[sat['id']][ip] = pneighbors['id'][argmax_take] + take_score[argmax_take]
                    centers[sat['id']] = [ pneighbors['x'][argmax_all], pneighbors['y'][argmax_all], pneighbors['z'][argmax_all] ]

            pklsave(give_scores, f"./nh_give_scores_host{BGG['id']:04d}_new.pickle", overwrite=True)
            pklsave(take_scores, f"./nh_take_scores_host{BGG['id']:04d}_new.pickle", overwrite=True)
            uri.timer.verbose=1

        hparking = iLG['hparking']
        if(len(hparking)>0):
            uri.timer.verbose=0
            centers = {}
            members = {}
            all_scores = {}
            give_scores = {}
            take_scores = {}
            for ip, pout in tqdm( enumerate(pouts), total=len(pouts), desc=f"H[{key:04}]" ):
                psnap = snaps.get_snap(pout)
                pgals = uhmi.HaloMaker.load(psnap, galaxy=False)

                for sub in hparking:
                    if(sub['id'] in centers.keys()):
                        center = centers[sub['id']]
                    else:
                        center = [sub['x'], sub['y'], sub['z']]
                        centers[sub['id']] = center
                    my_member = get_members(sub, galaxy=False)
                    pneighbors = cut_box(pgals, *center, rrange)


                    if(len(pneighbors)==0): continue

                    give_score = np.zeros(len(pneighbors))
                    take_score = np.zeros(len(pneighbors))
                    for i, pg in enumerate(pneighbors):
                        pmember = get_members(pg, galaxy=False)
                        intersect = np.sum( isin(pmember, my_member, assume_unique=True) )
                        give_score[i] = intersect / len(my_member) / 2
                        take_score[i] = intersect / len(pmember) / 2
                    all_score = give_score * take_score
                    
                    argmax_all = np.argmax(all_score)
                    argmax_give = np.argmax(give_score)
                    argmax_take = np.argmax(take_score)
                    if(not sub['id'] in all_scores.keys()):
                        all_scores[sub['id']] = np.zeros(len(pouts))
                        give_scores[sub['id']] = np.zeros(len(pouts))
                        take_scores[sub['id']] = np.zeros(len(pouts))
                    
                    all_scores[sub['id']][ip] = pneighbors['id'][argmax_all] + all_score[argmax_all]
                    give_scores[sub['id']][ip] = pneighbors['id'][argmax_give] + give_score[argmax_give]
                    take_scores[sub['id']][ip] = pneighbors['id'][argmax_take] + take_score[argmax_take]
                    centers[sub['id']] = [ pneighbors['x'][argmax_all], pneighbors['y'][argmax_all], pneighbors['z'][argmax_all] ]

            pklsave(give_scores, f"./nh_give_dm_scores_host{BGG['id']:04d}_new.pickle", overwrite=True)
            pklsave(take_scores, f"./nh_take_dm_scores_host{BGG['id']:04d}_new.pickle", overwrite=True)
            uri.timer.verbose=1

## New format for all scores

In [13]:
# Sat Scores

In [14]:
if(not 'e' in filedone):
    take_dict = {}; give_dict={}
    tmp = pklload(f"{database}/befo231031/08_nh_scores.pickle")
    take1 = tmp['take']
    for ikey in take1.keys():
        for key in take1[ikey].keys():
            if(key in take_dict.keys()): continue
            take_dict[key] = take1[ikey][key]
    give1 = tmp['give']
    for ikey in give1.keys():
        for key in give1[ikey].keys():
            if(key in give_dict.keys()): continue
            give_dict[key] = give1[ikey][key]
    take2 = pklload(f"./befo231109/recalc11/08_nh_take_scores_host0011.pickle")
    for key in take2.keys():
        if(key in take_dict.keys()): continue
        take_dict[key] = take2[key]
    give2 = pklload(f"./befo231109/recalc11/08_nh_give_scores_host0011.pickle")
    for key in give2.keys():
        if(key in give_dict.keys()): continue
        give_dict[key] = give2[key]

    for ikey in LG.keys():
        fname = f"./nh_take_scores_host{ikey:04d}_new.pickle"
        if(os.path.exists(fname)):
            tmp = pklload(fname)
            for key in tmp.keys():
                if(key in take_dict.keys()): continue
                take_dict[key] = tmp[key]
        fname = f"./nh_give_scores_host{ikey:04d}_new.pickle"
        if(os.path.exists(fname)):
            tmp = pklload(fname)
            for key in tmp.keys():
                if(key in give_dict.keys()): continue
                give_dict[key] = tmp[key]

    tlen = len(take_dict.keys()); glen = len(give_dict.keys())
    assert tlen == glen
    pklsave(take_dict, f"{database}/take_score_sats.pickle", overwrite=True)
    pklsave(give_dict, f"{database}/give_score_sats.pickle", overwrite=True)
else:
    take_dict = pklload(f"{database}/take_score_sats.pickle")
    give_dict = pklload(f"{database}/give_score_sats.pickle")

In [15]:
if(not 'e' in filedone):
    take_dict = {}; give_dict={}
    tmp = pklload(f"{database}/befo231031/08_nh_dm_scores.pickle")
    take1 = tmp['take']
    for ikey in take1.keys():
        for key in take1[ikey].keys():
            if(key in take_dict.keys()): continue
            take_dict[key] = take1[ikey][key]
    give1 = tmp['give']
    for ikey in give1.keys():
        for key in give1[ikey].keys():
            if(key in give_dict.keys()): continue
            give_dict[key] = give1[ikey][key]
    take2 = pklload(f"./befo231109/recalc11/08_nh_take_dm_scores_host0011.pickle")
    for key in take2.keys():
        if(key in take_dict.keys()): continue
        take_dict[key] = take2[key]
    give2 = pklload(f"./befo231109/recalc11/08_nh_give_dm_scores_host0011.pickle")
    for key in give2.keys():
        if(key in give_dict.keys()): continue
        give_dict[key] = give2[key]

    for ikey in LG.keys():
        fname = f"./befo231109/dontmiss/nh_take_dm_scores_host{ikey:04d}_new.pickle"
        if(os.path.exists(fname)):
            tmp = pklload(fname)
            for key in tmp.keys():
                if(key in take_dict.keys()): continue
                take_dict[key] = tmp[key]
        fname = f"./befo231109/dontmiss/nh_give_dm_scores_host{ikey:04d}_new.pickle"
        if(os.path.exists(fname)):
            tmp = pklload(fname)
            for key in tmp.keys():
                if(key in give_dict.keys()): continue
                give_dict[key] = tmp[key]
        fname = f"./nh_take_dm_scores_host{ikey:04d}_new.pickle"
        if(os.path.exists(fname)):
            tmp = pklload(fname)
            for key in tmp.keys():
                if(key in take_dict.keys()): continue
                take_dict[key] = tmp[key]
        fname = f"./nh_give_dm_scores_host{ikey:04d}_new.pickle"
        if(os.path.exists(fname)):
            tmp = pklload(fname)
            for key in tmp.keys():
                if(key in give_dict.keys()): continue
                give_dict[key] = tmp[key]

    tlen = len(take_dict.keys()); glen = len(give_dict.keys())
    assert tlen == glen
    pklsave(take_dict, f"{database}/take_score_subs.pickle", overwrite=True)
    pklsave(give_dict, f"{database}/give_score_subs.pickle", overwrite=True)
else:
    take_dict = pklload(f"{database}/take_score_subs.pickle")
    give_dict = pklload(f"{database}/give_score_subs.pickle")

## Filter by scores

In [16]:
if(not 'e' in filedone):
    for key in LG.keys():
        print(f"[{key:04d}]")
        BGG = LG[key]['BGG']
        sats = LG[key]['Final_sats']
        subs = LG[key]['Final_subs']
        real = LG[key]['Final_real']

        #-------------------------------------------------------
        # CONFIRMED
        #-------------------------------------------------------
        # scores = pklload(f"{database}/take_score_sats.pickle")
        # for satid in sats:
        #     assert satid in scores.keys()
        
        # scores = pklload(f"{database}/take_score_subs.pickle")
        # for subid in subs:
        #     assert subid in scores.keys()
        #-------------------------------------------------------
        sat_banlist = []
        scores = pklload(f"{database}/take_score_sats.pickle")
        for satid in sats:
            isgood = np.median(scores[satid]%1) > 0.15
            if(not isgood):
                sat_banlist.append(satid)
        newsats = sats[~isin(sats, sat_banlist)]
        
        sub_banlist = []
        scores = pklload(f"{database}/take_score_subs.pickle")
        for subid in subs:
            isgood = np.median(scores[subid]%1) > 0.15
            if(not isgood):
                sub_banlist.append(subid)
        newsubs = subs[~isin(subs, sub_banlist)]

        newreal = np.copy(real)
        for i in range(len(newreal)):
            if(newreal[i]['state'] == 'pair'):
                if(not newreal[i]['gid'] in newsats):
                    newsats = np.append(newsats,newreal[i]['gid'])
                if(not newreal[i]['hid'] in newsubs):
                    newsubs = np.append(newsubs,newreal[i]['hid'])
            elif(newreal[i]['state'] == 'dink'):
                if(not newreal[i]['hid'] in newsubs):
                    newreal[i]['state'] = 'ban'
            elif(newreal[i]['state'] == 'orphan'):
                if(not newreal[i]['gid'] in newsats):
                    newreal[i]['state'] = 'ban'
        print(f"G{len(newsats)}/{len(sats)} | H{len(newsubs)}/{len(subs)} | R{np.sum(newreal['state']!='ban')}/{len(newreal)}")
        LG[key]['Final_sats'] = newsats
        LG[key]['Final_subs'] = newsubs
        LG[key]['Final_real'] = newreal
    pklsave(LG, f"{database}/00_LocalGroup_final_e_removeunreal.pickle", overwrite=True)

# Make Catalogs

In [18]:
sat_dtype = LG[1]['sats'].dtype
sat_dtype.names

('nparts',
 'id',
 'timestep',
 'level',
 'host',
 'hostsub',
 'nbsub',
 'nextsub',
 'aexp',
 'm',
 'x',
 'y',
 'z',
 'vx',
 'vy',
 'vz',
 'Lx',
 'Ly',
 'Lz',
 'r',
 'a',
 'b',
 'c',
 'ek',
 'ep',
 'et',
 'spin',
 'sigma',
 'sigma_bulge',
 'm_bulge',
 'rvir',
 'mvir',
 'tvir',
 'cvel',
 'rho0',
 'rc',
 'halo_id',
 'halo_nparts',
 'halo_level',
 'halo_host',
 'halo_hostsub',
 'halo_x',
 'halo_y',
 'halo_z',
 'halo_vx',
 'halo_vy',
 'halo_vz',
 'halo_mvir',
 'halo_rvir',
 'fcontam',
 'dist',
 'central',
 'main',
 'Host',
 'r50m',
 'r90m',
 'r50r',
 'r90r',
 'SFR_mem',
 'u_mem',
 'g_mem',
 'r_mem',
 'i_mem',
 'z_mem',
 'metal_mem',
 'ager_mem',
 't50_mem',
 't90_mem',
 'SFR_r50m',
 'u_r50m',
 'g_r50m',
 'r_r50m',
 'i_r50m',
 'z_r50m',
 'metal_r50m',
 'ager_r50m',
 't50_r50m',
 't90_r50m',
 'mgas_r50m',
 'mcold_r50m',
 'mdm_r50m',
 'SFR_r90m',
 'u_r90m',
 'g_r90m',
 'r_r90m',
 'i_r90m',
 'z_r90m',
 'metal_r90m',
 'ager_r90m',
 't50_r90m',
 't90_r90m',
 'mgas_r90m',
 'mcold_r90m',
 'mdm_r90

In [19]:
cols = [
    "Host", "Sat", "r50m", "r90m", "r50r", "r90r", 
    "SFR_mem", "u_mem", "g_mem", "r_mem", "i_mem", "z_mem", "metal_mem", "ager_mem", "t50_mem", "t90_mem"] 
category = ["r50m", "r90m", "r50r", "r90r", "rmax"]


for icate in category:
    cols = cols+[f"SFR_{icate}", f"u_{icate}", f"g_{icate}", f"r_{icate}", f"i_{icate}", f"z_{icate}", f"metal_{icate}", f"ager_{icate}", f"t50_{icate}", f"t90_{icate}", f"mgas_{icate}", f"mcold_{icate}", f"mdm_{icate}"]

def calc_rhalf(gal, part, weights, ratio=0.5):
    dist = distance(gal, part)
    argsort = np.argsort(dist)
    sw = np.cumsum(weights[argsort])
    sw /= sw[-1]
    return dist[argsort][np.argmin(np.abs(sw-ratio))]

def calc_rhalf_sorted(sorted_dist, sorted_weights, ratio=0.5):
    sw = np.cumsum(sorted_weights)
    sw /= sw[-1]
    return sorted_dist[np.argmin(np.abs(sw-ratio))]

def calc_tform(part, weights, ratio=0.5):
    age = part['age','Gyr']
    argsort = np.argsort(age)
    sw = np.cumsum(weights[argsort])
    sw /= sw[-1]
    return age[argsort][np.argmin(np.abs(sw-ratio))]

`make_gcatalog`

In [29]:
def make_gcatalog(Hostkey:int, table:np.void, istar:uri.Particle, idm:uri.Particle, icell:uri.Cell) -> np.void:
    global snap, sat_dtype
    result = np.zeros(1, dtype=sat_dtype)[0]
    for iname in result.dtype.names:
        if(iname in table.dtype.names):
            result[iname] = table[iname]
    result['Host'] = Hostkey

    satid = table['id']
    pid = uhmi.HaloMaker.read_member_part(snap, satid, galaxy=True, simple=True).flatten()
    ind = isin(np.abs(istar['id']), pid)
    ibox = np.array([
        [table['x']-table['r'], table['x']+table['r']],
        [table['y']-table['r'], table['y']+table['r']],
        [table['z']-table['r'], table['z']+table['r']]
                     ])
    assert np.sum(ind) == len(pid), f"{np.sum(ind)}, {len(pid)}\n{snap.box} <-> ({ibox})"
    mem_star = istar[ind]
    mem_dist = distance(table, mem_star)
    argsort = np.argsort(mem_dist)
    mem_dist = mem_dist[argsort]
    mem_mass = mem_star['m'][argsort]
    mem_rband = measure_luminosity(mem_star, 'SDSS_r', model='cb07')[argsort]

    result['r50m'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.5)
    result['r90m'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.9)
    result['r50r'] = calc_rhalf_sorted(mem_dist, mem_rband, ratio=0.5)
    result['r90r'] = calc_rhalf_sorted(mem_dist, mem_rband, ratio=0.9)

    ind = mem_star['age', 'Myr'] < 100
    result['SFR_mem'] = np.sum(mem_star['m', 'Msol'][ind]) / 1e8
    result['u_mem'] = measure_luminosity(mem_star, 'SDSS_u', model='cb07', total=True)
    result['g_mem'] = measure_luminosity(mem_star, 'SDSS_g', model='cb07', total=True)
    result['r_mem'] = measure_luminosity(mem_star, 'SDSS_r', model='cb07', total=True)
    result['i_mem'] = measure_luminosity(mem_star, 'SDSS_i', model='cb07', total=True)
    result['z_mem'] = measure_luminosity(mem_star, 'SDSS_z', model='cb07', total=True)
    result['metal_mem'] = np.sum(mem_star['metal'] * mem_star['m']) / np.sum(mem_star['m'])
    result['ager_mem'] = np.average(mem_star['age', 'Gyr'], weights=mem_rband)
    result['t50_mem'] = calc_tform(mem_star, mem_rband, ratio=0.5)
    result['t90_mem'] = calc_tform(mem_star, mem_rband, ratio=0.9)

    radiis = [result['r50m'], result['r90m'], result['r50r'], result['r90r'], table['r']]
    for radii, rname in zip(radiis, category):
        cut_star = cut_sphere(istar, table['x'], table['y'], table['z'], radii)
        rband = measure_luminosity(cut_star, 'SDSS_r', model='cb07')
        result[f'SFR_{rname}'] = np.sum(cut_star['m', 'Msol'][cut_star['age', 'Myr'] < 100]) / 1e8
        result[f'u_{rname}'] = measure_luminosity(cut_star, 'SDSS_u', model='cb07', total=True)
        result[f'g_{rname}'] = measure_luminosity(cut_star, 'SDSS_g', model='cb07', total=True)
        result[f'r_{rname}'] = measure_luminosity(cut_star, 'SDSS_r', model='cb07', total=True)
        result[f'i_{rname}'] = measure_luminosity(cut_star, 'SDSS_i', model='cb07', total=True)
        result[f'z_{rname}'] = measure_luminosity(cut_star, 'SDSS_z', model='cb07', total=True)
        result[f'metal_{rname}'] = np.sum(cut_star['metal'] * cut_star['m']) / np.sum(cut_star['m'])
        result[f'ager_{rname}'] = np.average(cut_star['age', 'Gyr'], weights=rband)
        result[f't50_{rname}'] = calc_tform(cut_star, rband, ratio=0.5)
        result[f't90_{rname}'] = calc_tform(cut_star, rband, ratio=0.9)
        cut_gas = cut_sphere(icell, table['x'], table['y'], table['z'], radii)
        cut_dm = cut_sphere(idm, table['x'], table['y'], table['z'], radii)
        result[f'mgas_{rname}'] = np.sum(cut_gas['m', 'Msol'])
        coldind = cut_gas['T', 'K'] < 1e4
        result[f'mcold_{rname}'] = np.sum(cut_gas['m', 'Msol'][coldind])
        result[f'mdm_{rname}'] = np.sum(cut_dm['m', 'Msol'])

    return result

In [21]:
sub_dtype = LG[1]['subs'].dtype
sub_dtype = np.dtype(sub_dtype.descr + [('r200kpc', '<f8'), ('m200', '<f8'), ('r200', '<f8')])
sub_dtype

dtype([('nparts', '<i4'), ('id', '<i4'), ('timestep', '<i4'), ('level', '<i4'), ('host', '<i4'), ('hostsub', '<i4'), ('nbsub', '<i4'), ('nextsub', '<i4'), ('aexp', '<f8'), ('m', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Lx', '<f8'), ('Ly', '<f8'), ('Lz', '<f8'), ('r', '<f8'), ('a', '<f8'), ('b', '<f8'), ('c', '<f8'), ('ek', '<f8'), ('ep', '<f8'), ('et', '<f8'), ('spin', '<f8'), ('sigma', '<f8'), ('rvir', '<f8'), ('mvir', '<f8'), ('tvir', '<f8'), ('cvel', '<f8'), ('rho0', '<f8'), ('rc', '<f8'), ('mcontam', '<f8'), ('r10_mem', '<f8'), ('r50_mem', '<f8'), ('r90_mem', '<f8'), ('r10_vir', '<f8'), ('r50_vir', '<f8'), ('r90_vir', '<f8'), ('r10_max', '<f8'), ('r50_max', '<f8'), ('r90_max', '<f8'), ('sub', '<i8'), ('dink', '?'), ('mdm', '<f8'), ('mstar', '<f8'), ('mcold', '<f8'), ('mcell', '<f8'), ('Host', '<i4'), ('r200kpc', '<f8'), ('m200', '<f8'), ('r200', '<f8')])

`make_hcatalog`

In [22]:
def make_hcatalog(Hostkey:int, table:np.void, istar:uri.Particle, idm:uri.Particle, icell:uri.Cell) -> np.void:
    global snap, sub_dtype, database
    assert 'm200' in table.dtype.names
    result = np.zeros(1, dtype=sub_dtype)[0]
    for iname in result.dtype.names:
        if(iname in table.dtype.names):
            result[iname] = table[iname]

    result['Host'] = Hostkey
    result['mdm'] = np.sum(idm['m','Msol'])
    result['mstar'] = np.sum(istar['m','Msol'])
    result['mcell'] = np.sum(icell['m','Msol'])
    mask = icell['T','K'] < 2e4
    if(True in mask):
        result['mcold'] = np.sum(icell['m','Msol'][mask])

    all_dist = distance(table, idm); argsort = np.argsort(all_dist)
    all_dist = all_dist[argsort]; all_mass = idm['m'][argsort]
    memdm = uhmi.HaloMaker.read_member_part(snap, table['id'], galaxy=False, target_fields=['x','y','z','m'])
    mem_dist = distance(table, memdm); argsort = np.argsort(mem_dist)
    mem_dist = mem_dist[argsort]; mem_mass = memdm['m'][argsort]

    result['r10_mem'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.1)
    result['r50_mem'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.5)
    result['r90_mem'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.9)
    result['r10_max'] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.1)
    result['r50_max'] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.5)
    result['r90_max'] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.9)
    virdm, ind = cut_sphere(idm, table['x'], table['y'], table['z'], table['rvir'], return_index=True)
    all_dist = all_dist[ind]; all_mass = idm['m'][ind]
    result['r10_vir'] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.1)
    result['r50_vir'] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.5)
    result['r90_vir'] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.9)
    
    return result
    

main routine

In [33]:
print( newsubs.dtype.names, oldsubs.dtype.names )

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'mcontam', 'r10_mem', 'r50_mem', 'r90_mem', 'r10_vir', 'r50_vir', 'r90_vir', 'r10_max', 'r50_max', 'r90_max', 'sub', 'dink', 'mdm', 'mstar', 'mcold', 'mcell', 'Host', 'r200kpc', 'm200', 'r200') ('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'mcontam', 'r10_mem', 'r50_mem', 'r90_mem', 'r10_vir', 'r50_vir', 'r90_vir', 'r10_max', 'r50_max', 'r90_max', 'sub', 'dink', 'mdm', 'mstar', 'mcold', 'mcell', 'Host')


In [53]:
def box_in_box(sbox, lbox):
    return (sbox[0,0] >= lbox[0,0]) & (sbox[0,1] <= lbox[0,1]) & (sbox[1,0] >= lbox[1,0]) & (sbox[1,1] <= lbox[1,1]) & (sbox[2,0] >= lbox[2,0]) & (sbox[2,1] <= lbox[2,1])

In [56]:
star.dtype

dtype([('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('m', '<f8'), ('epoch', '<f8'), ('metal', '<f8'), ('id', '<i4'), ('level', '<i4'), ('cpu', '<i4')])

In [54]:
if(not 'f' in filedone):
    uri.timer.verbose=1
    for key in LG.keys():
        print(f"[{key:04d}]")
        BGG = LG[key]['BGG']
        oldsats = LG[key]['sats']
        oldsubs = LG[key]['subs']
        satids = LG[key]['Final_sats']
        subids = LG[key]['Final_subs']
        
        newsats = np.zeros(len(satids), dtype=sat_dtype)
        newsubs = np.zeros(len(subids), dtype=sub_dtype)
        newreal = LG[key]['Final_real']

        snap.box = LG[key]['box']

        # Box check
        gtmp = gals[satids-1]; htmp = nhals[subids-1]
        x1 = min( np.min(gtmp['x'] - 1.1*gtmp['r']), np.min(htmp['x'] - 1.1*htmp['r']), BGG['x'] - 1.5*BGG['r200_code'] )
        x2 = max( np.max(gtmp['x'] + 1.1*gtmp['r']), np.max(htmp['x'] + 1.1*htmp['r']), BGG['x'] + 1.5*BGG['r200_code'] )
        y1 = min( np.min(gtmp['y'] - 1.1*gtmp['r']), np.min(htmp['y'] - 1.1*htmp['r']), BGG['y'] - 1.5*BGG['r200_code'] )
        y2 = max( np.max(gtmp['y'] + 1.1*gtmp['r']), np.max(htmp['y'] + 1.1*htmp['r']), BGG['y'] + 1.5*BGG['r200_code'] )
        z1 = min( np.min(gtmp['z'] - 1.1*gtmp['r']), np.min(htmp['z'] - 1.1*htmp['r']), BGG['z'] - 1.5*BGG['r200_code'] )
        z2 = max( np.max(gtmp['z'] + 1.1*gtmp['r']), np.max(htmp['z'] + 1.1*htmp['r']), BGG['z'] + 1.5*BGG['r200_code'] )
        sbox = np.array([[x1,x2],[y1,y2],[z1,z2]])

        if(box_in_box(sbox, snap.box)):
            print("Box check passed -> Shrink")
            LG[key]['box'] = sbox
            star = pklload(f"{database}/parts/nh_star_{key:04d}.pickle")
            mask = (star['x'] >= sbox[0,0]) & (star['x'] <= sbox[0,1]) & (star['y'] >= sbox[1,0]) & (star['y'] <= sbox[1,1]) & (star['z'] >= sbox[2,0]) & (star['z'] <= sbox[2,1])
            pklsave(star[mask], f"{database}/parts/nh_star_{key:04d}.pickle", overwrite=True)
            dm = pklload(f"{database}/parts/nh_dm_{key:04d}.pickle")
            mask = (dm['x'] >= sbox[0,0]) & (dm['x'] <= sbox[0,1]) & (dm['y'] >= sbox[1,0]) & (dm['y'] <= sbox[1,1]) & (dm['z'] >= sbox[2,0]) & (dm['z'] <= sbox[2,1])
            pklsave(dm[mask], f"{database}/parts/nh_dm_{key:04d}.pickle", overwrite=True)
            cell = pklload(f"{database}/parts/nh_cell_{key:04d}.pickle")
            mask = (cell['x'] >= sbox[0,0]) & (cell['x'] <= sbox[0,1]) & (cell['y'] >= sbox[1,0]) & (cell['y'] <= sbox[1,1]) & (cell['z'] >= sbox[2,0]) & (cell['z'] <= sbox[2,1])
            pklsave(cell[mask], f"{database}/parts/nh_cell_{key:04d}.pickle", overwrite=True)
        else:
            print("Box check failed -> Recalc box")
            print(sbox)
            print(snap.box)
            snap.box = sbox; LG[key]['box']=sbox
            print("Load particles")
            snap.get_part(nthread=32)
            pklsave(snap.part['star'].table, f"{database}/parts/nh_star_{key:04d}.pickle", overwrite=True)
            pklsave(snap.part['dm'].table, f"{database}/parts/nh_dm_{key:04d}.pickle", overwrite=True)
            snap.part=None; snap.part_data=None, snap.cpulist_part=[]
            print("Load cells")
            snap.get_cell(nthread=32)
            pklsave(snap.cell.table, f"{database}/parts/nh_cell_{key:04d}.pickle", overwrite=True)
            snap.cell=None; snap.cell_data=None, snap.cpulist_cell=[]
    uri.timer.verbose=0
    pklsave(LG, f"{database}/00_LocalGroup_final_f_newbox.pickle", overwrite=True)

[0001]
Box check failed, recalc box
[[0.46326325 0.47044213]
 [0.47227537 0.4795758 ]
 [0.51605117 0.52295538]]
[[0.46292417 0.47066612]
 [0.47213055 0.47978913]
 [0.51581757 0.52274353]]


ValueError: Box check failed

In [34]:
if(not 'g' in filedone):
    for key in LG.keys():
        print(f"[{key:04d}]")
        BGG = LG[key]['BGG']
        oldsats = LG[key]['sats']
        oldsubs = LG[key]['subs']
        satids = LG[key]['Final_sats']
        subids = LG[key]['Final_subs']
        
        newsats = np.zeros(len(satids), dtype=sat_dtype)
        newsubs = np.zeros(len(subids), dtype=sub_dtype)
        newreal = LG[key]['Final_real']

        snap.box = LG[key]['box']
        star = uri.Particle(pklload(f"{database}/parts/nh_star_{key:04d}.pickle"), snap)
        dm = uri.Particle(pklload(f"{database}/parts/nh_dm_{key:04d}.pickle"), snap)
        cell = uri.Cell(pklload(f"{database}/parts/nh_cell_{key:04d}.pickle"), snap)

        # Box check
        gtmp = gals[satids-1]; htmp = nhals[subids-1]
        x1 = min( np.min(gtmp['x'] - 1.1*gtmp['r']), np.min(htmp['x'] - 1.1*htmp['r']), BGG['x'] - 1.5*BGG['r200_code'] )
        x2 = max( np.max(gtmp['x'] + 1.1*gtmp['r']), np.max(htmp['x'] + 1.1*htmp['r']), BGG['x'] + 1.5*BGG['r200_code'] )
        y1 = min( np.min(gtmp['y'] - 1.1*gtmp['r']), np.min(htmp['y'] - 1.1*htmp['r']), BGG['y'] - 1.5*BGG['r200_code'] )
        y2 = max( np.max(gtmp['y'] + 1.1*gtmp['r']), np.max(htmp['y'] + 1.1*htmp['r']), BGG['y'] + 1.5*BGG['r200_code'] )
        z1 = min( np.min(gtmp['z'] - 1.1*gtmp['r']), np.min(htmp['z'] - 1.1*htmp['r']), BGG['z'] - 1.5*BGG['r200_code'] )
        z2 = max( np.max(gtmp['z'] + 1.1*gtmp['r']), np.max(htmp['z'] + 1.1*htmp['r']), BGG['z'] + 1.5*BGG['r200_code'] )
        sbox = np.array([[x1,x2],[y1,y2],[z1,z2]])
        if(box_in_box(sbox, snap.box)):
            print("Box check passed")
        else:
            print("Box check failed, recalc box")
            print(sbox)
            print(snap.box)
            raise ValueError("Box check failed")

        for i, satid in tqdm( enumerate(satids), total=len(satids), desc=f"G[{key:04}]"):
            if(satid in oldsats['id']): newsats[i] = oldsats[oldsats['id'] == satid][0]
            else:
                sat = gals[satid-1]
                istar = cut_sphere(star, sat['x'], sat['y'], sat['z'], 1.5*sat['r'])
                idm = cut_sphere(dm, sat['x'], sat['y'], sat['z'], 1.5*sat['r'])
                icell = cut_sphere(cell, sat['x'], sat['y'], sat['z'], 1.5*sat['r'])

                newcat = make_gcatalog(key, sat, istar, idm, icell)
                newsats[i] = newcat
        
        for i, subid in tqdm( enumerate(subids), total=len(subids), desc=f"H[{key:04}]" ):
            sub = nhals[subid-1]
            if(subid in oldsubs['id']):
                old = oldsubs[oldsubs['id'] == subid][0]
                for iname in newsubs.dtype.names:
                    if(iname in old.dtype.names): newsubs[i][iname] = old[iname]
                    elif(iname in sub.dtype.names): newsubs[i][iname] = sub[iname]
                    else: raise TypeError(f"`{iname}` is not in sub dtype")
            else:
                fname = f"{database}/parts/insub/nh_dm_{key:04d}_{sub['id']:07d}.pickle"
                if(os.path.exists(fname)): idm = uri.Particle(pklload(fname), snap)
                else:
                    idm = cut_sphere(dm, sub['x'], sub['y'], sub['z'], sub['r'])
                    pklsave(idm.table, fname, overwrite=True)
                fname = f"{database}/parts/insub/nh_star_{key:04d}_{sub['id']:07d}.pickle"
                if(os.path.exists(fname)): istar = uri.Particle(pklload(fname), snap)
                else:
                    istar = cut_sphere(star, sub['x'], sub['y'], sub['z'], sub['r'])
                    pklsave(istar.table, fname, overwrite=True)
                fname = f"{database}/parts/insub/nh_cell_{key:04d}_{sub['id']:07d}.pickle"
                if(os.path.exists(fname)): icell = uri.Cell(pklload(fname), snap)
                else:
                    icell = cut_sphere(cell, sub['x'], sub['y'], sub['z'], sub['r'])
                    pklsave(icell.table, fname, overwrite=True)

                newsub = make_hcatalog(key, sub, istar, idm, icell)
                newsubs[i] = newsub
        
        LG[key]['sats'] = newsats
        LG[key]['subs'] = newsubs
        LG[key]['real'] = newreal[newreal['state'] != 'ban']

    pklsave(LG, f"{database}/00_LocalGroup_final_g_addcat.pickle", overwrite=True)
    filedone.append('g')    

[0001]


H[0001]: 100%|██████████| 140/140 [00:39<00:00,  3.58it/s]


[0002]


H[0002]:  59%|█████▉    | 495/835 [13:53<09:32,  1.68s/it]


KeyboardInterrupt: 

In [ ]:
filedone.append('g')

# Find UDG

In [59]:
def make_udg(host:int, istar_vir:uri.Particle, idm_vir:uri.Particle, icell_vir:uri.Cell) -> np.void:
    global snap, sat_dtype
    result = np.zeros(1, dtype=sat_dtype)[0]
    names = ['timestep', 'aexp', 'Host']
    for name in names:
        result[name] = host[name]
    result['id'] = -host['id']
    result['fcontam'] = host['mcontam']/host['m']
    result['nparts'] = len(istar_vir)
    names = ['halo_id', 'halo_nparts', 'halo_level', 'halo_host', 'halo_hostsub', 'halo_x', 'halo_y', 'halo_z', 'halo_vx', 'halo_vy', 'halo_vz', 'halo_mvir', 'halo_rvir']
    for name in names:
        result[name] = host[name[5:]]
    # Default Info
    result['m'] = np.sum(istar_vir['m', 'Msol'])
    result['x'] = np.average(istar_vir['x'], weights=istar_vir['m'])
    result['y'] = np.average(istar_vir['y'], weights=istar_vir['m'])
    result['z'] = np.average(istar_vir['z'], weights=istar_vir['m'])
    result['vx'] = np.average(istar_vir['vx','km/s'], weights=istar_vir['m'])
    result['vy'] = np.average(istar_vir['vy','km/s'], weights=istar_vir['m'])
    result['vz'] = np.average(istar_vir['vz','km/s'], weights=istar_vir['m'])
    result['dist'] = distance(result, host)
    # Calculate the angular momentum
    dx = istar_vir['x'] - result['x']; dx /= snap.unit['Mpc']
    dy = istar_vir['y'] - result['y']; dy /= snap.unit['Mpc']
    dz = istar_vir['z'] - result['z']; dz /= snap.unit['Mpc']
    dpx = (istar_vir['vx','km/s'] - result['vx'])*istar_vir['m','Msol']
    dpy = (istar_vir['vy','km/s'] - result['vy'])*istar_vir['m','Msol']
    dpz = (istar_vir['vz','km/s'] - result['vz'])*istar_vir['m','Msol']
    result['Lx'] = np.sum(dpy*dz - dpz*dy)/1e11 # in 10**11 Msun * km/s * Mpc
    result['Ly'] = np.sum(dpz*dx - dpx*dz)/1e11
    result['Lz'] = np.sum(dpx*dy - dpy*dx)/1e11
    result['r'] = np.max(np.sqrt(dx**2 + dy**2 + dz**2)) * snap.unit['Mpc']
    # Value Added
    mem_dist = distance(result, istar_vir)
    argsort = np.argsort(mem_dist)
    mem_dist = mem_dist[argsort]
    mem_mass = istar_vir['m'][argsort]
    mem_rband = measure_luminosity(istar_vir, 'SDSS_r', model='cb07')[argsort]
    result['r50m'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.5)
    result['r90m'] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.9)
    result['r50r'] = calc_rhalf_sorted(mem_dist, mem_rband, ratio=0.5)
    result['r90r'] = calc_rhalf_sorted(mem_dist, mem_rband, ratio=0.9)
    # Use member particles
    ind = istar_vir['age', 'Myr'] < 100
    result['SFR_mem'] = np.sum(istar_vir['m', 'Msol'][ind]) / 1e8
    result['u_mem'] = measure_luminosity(istar_vir, 'SDSS_u', model='cb07', total=True)
    result['g_mem'] = measure_luminosity(istar_vir, 'SDSS_g', model='cb07', total=True)
    result['r_mem'] = measure_luminosity(istar_vir, 'SDSS_r', model='cb07', total=True)
    result['i_mem'] = measure_luminosity(istar_vir, 'SDSS_i', model='cb07', total=True)
    result['z_mem'] = measure_luminosity(istar_vir, 'SDSS_z', model='cb07', total=True)
    result['metal_mem'] = np.sum(istar_vir['metal'] * istar_vir['m']) / np.sum(istar_vir['m'])
    result['ager_mem'] = np.average(istar_vir['age', 'Gyr'], weights=rband)
    result['t50_mem'] = calc_tform(istar_vir, rband, ratio=0.5)
    result['t90_mem'] = calc_tform(istar_vir, rband, ratio=0.9)
    radiis = [result['r50m'], result['r90m'], result['r50r'], result['r90r'], result['r']]
    for radii, rname in zip(radiis, category):
        cut_star, cutind = cut_sphere(istar_vir, result['x'], result['y'], result['z'], radii, return_index=True)
        rband = measure_luminosity(cut_star, 'SDSS_r', model='cb07')
        result[f'SFR_{rname}'] = np.sum(cut_star['m', 'Msol'][cut_star['age', 'Myr'] < 100]) / 1e8
        result[f'u_{rname}'] = measure_luminosity(cut_star, 'SDSS_u', model='cb07', total=True)
        result[f'g_{rname}'] = measure_luminosity(cut_star, 'SDSS_g', model='cb07', total=True)
        result[f'r_{rname}'] = measure_luminosity(cut_star, 'SDSS_r', model='cb07', total=True)
        result[f'i_{rname}'] = measure_luminosity(cut_star, 'SDSS_i', model='cb07', total=True)
        result[f'z_{rname}'] = measure_luminosity(cut_star, 'SDSS_z', model='cb07', total=True)
        result[f'metal_{rname}'] = np.sum(cut_star['metal'] * cut_star['m']) / np.sum(cut_star['m'])
        result[f'ager_{rname}'] = np.average(cut_star['age', 'Gyr'], weights=rband)
        result[f't50_{rname}'] = calc_tform(cut_star, rband, ratio=0.5)
        result[f't90_{rname}'] = calc_tform(cut_star, rband, ratio=0.9)
        cut_gas = cut_sphere(icell, result['x'], result['y'], result['z'], radii)
        cut_dm = cut_sphere(idm, result['x'], result['y'], result['z'], radii)
        result[f'mgas_{rname}'] = np.sum(cut_gas['m', 'Msol'])
        coldind = cut_gas['T', 'K'] < 1e4
        result[f'mcold_{rname}'] = np.sum(cut_gas['m', 'Msol'][coldind])
        result[f'mdm_{rname}'] = np.sum(cut_dm['m', 'Msol'])

    return result

In [52]:
if(not 'h' in filedone):
    for key in LG.keys():
        print(f"[{key:04d}]")
        BGG = LG[key]['BGG']
        sats = LG[key]['sats']
        subs = LG[key]['subs']
        real = LG[key]['real']
        ind = subs['mstar'] <= 1e3; subs[ind]['mstar'] = 0
        ind = subs['mcell'] <= 1e3; subs[ind]['mcell'] = 0
        ind = subs['mcold'] <= 1e3; subs[ind]['mcold'] = 0
        LG[key]['subs'] = subs
        LG[key]['UDG'] = []
        igals = gals[~isin(gals['id'], sats['id'])]
        igals = igals[igals['id'] != BGG['id']]


        dinks = real[real['gid'] < 0]['hid']

        star = uri.Particle(pklload(f"{database}/parts/nh_star_{key:04d}.pickle"), snap)
        dm = uri.Particle(pklload(f"{database}/parts/nh_dm_{key:04d}.pickle"), snap)
        cell = uri.Cell(pklload(f"{database}/parts/nh_cell_{key:04d}.pickle"), snap)

        newsats = np.copy(sats); newreal = np.copy(real)
        for i,dink in enumerate(dinks):
            sub = subs[subs['id'] == dink][0]
            if(sub['mstar'] < 6e5): continue

            insides1 = sphere_touch_sphere(sub, igals, r1='rvir', factor=0.75)
            insides2 = igals['r'] < 2*sub['rvir']
            insides3 = ~isin(igals['id'], newreal['gid'])
            insides = insides1&insides2&insides3
            gcands = igals[insides]
            if(len(gcands)==0):
                # Truly No gals in this sub
                if(len(star)>300):
                    LG[key]['UDG'].append(dink)   # <-------------- Find UDG candidates!
                continue
            elif(len(gcands)==1):
                # One son sat in this sub
                gcand = gcands[0]
                print(f"\t[{sub['id']}] 1 son")
            else:
                # Multiple son sats in this sub
                dist = distance(gcands, sub)
                argmin = np.argmin(dist)
                gcand = gcands[argmin]
                print(f"\t[{sub['id']}] {len(gcands)} sons")

            # If you reach here, then you must find `gcand`
            if(gcand['id'] in real['gid']):
                # Connect `sub` and `gcand`
                assert gcand['id'] in sats['id']
                hwhere = np.where(newreal['hid'] == sub['id'])[0][0]
                gwhere = np.where(newreal['gid'] == gcand['id'])[0][0]
                newreal[hwhere]['gid'] = gcand['id']
                newreal[hwhere]['state'] = 'pair'
                newreal[gwhere]['state'] = 'ban'
            else:
                # Make new `gcand`
                istar = cut_sphere(star, gcand['x'], gcand['y'], gcand['z'], 1.5*gcand['r'])
                idm = cut_sphere(dm, gcand['x'], gcand['y'], gcand['z'], 1.5*gcand['r'])
                icell = cut_sphere(cell, gcand['x'], gcand['y'], gcand['z'], 1.5*gcand['r'])
                newcat = make_gcatalog(key, gcand, istar, idm, icell)
                newsats = np.append(newsats, newcat)

                hwhere = np.where(newreal['hid'] == sub['id'])[0][0]
                newreal[hwhere]['gid'] = gcand['id']
                newreal[hwhere]['state'] = 'pair'

        if(len(LG[key]['UDG']) > 0):
            for udghostid in LG[key]['UDG']:
                udghost = subs[subs['id'] == udghostid][0]

                istar_vir = cut_sphere(star, udghost['x'], udghost['y'], udghost['z'], udghost['rvir'])
                if(len(istar_vir)>=100):
                    idm_vir = cut_sphere(dm, udghost['x'], udghost['y'], udghost['z'], udghost['rvir'])
                    icell_vir = cut_sphere(cell, udghost['x'], udghost['y'], udghost['z'], udghost['rvir'])
                    udg = make_udg(udghost, istar_vir:uri.Particle, idm_vir:uri.Particle, icell_vir:uri.Cell)
                    
                    newsats = np.append(newsats, udg)
                    hwhere = np.where(newreal['hid'] == udghost['id'])[0][0]
                    newreal[hwhere]['gid'] = udg['id']
                    newreal[hwhere]['state'] = 'upair'
        
        LG[key]['sats'] = newsats
        LG[key]['real'] = newreal[newreal['state'] != 'ban']
    pklsave(LG, f"{database}/00_LocalGroup_final_h_addudg.pickle", overwrite=True)

[0001]
[0002]
[0003]
[0004]
[0007]
[0008]
[0009]
[0010]
[0014]
[0017]
[0023]
[0024]
[0025]
[0026]
[0029]
[0035]
[0036]
[0037]
[0040]
[0046]
[0047]
[0052]
[0063]
[0064]
[0075]
[0084]
[0107]
[0108]
[0121]
[0129]
[0207]
[0011]


In [38]:
LG[key]['Final_real']

array([( 5493,   -1, 'dink'), (24608,   -1, 'dink'),
       (24611,   -1, 'dink'), (24612,   -1, 'ban'), (24613,   -1, 'dink'),
       (24615,   -1, 'dink'), (24616,   -1, 'dink'), (24623,   -1, 'ban'),
       (24624,   -1, 'ban'), (24627,   -1, 'ban'), (24628,   -1, 'dink'),
       (24629,   -1, 'ban'), (24630,   -1, 'dink'), (24632,   -1, 'dink'),
       (24633,   -1, 'dink'), (24634,   -1, 'ban'), (24635,   -1, 'dink'),
       (24636,   -1, 'dink'), (24639,   -1, 'dink'), (24640,   -1, 'ban'),
       (24642,   -1, 'dink'), (24644,   -1, 'dink'), (24645,   -1, 'ban'),
       (24647,   -1, 'ban'), (24648,   -1, 'ban'), (24649,  326, 'pair'),
       (24650,   -1, 'dink'), (24651,   -1, 'dink'),
       (24652,  513, 'pair'), (24653,   -1, 'dink'), (24654,   -1, 'ban'),
       (24655,   -1, 'ban'), (24656,   -1, 'dink'), ( 5661,   -1, 'dink'),
       (24553,   -1, 'ban'), (24562,   -1, 'dink'), (24574,   -1, 'dink'),
       (24593,   -1, 'dink'), (24601,   -1, 'dink'),
       (24604,   -